### tes Producer

In [3]:
from confluent_kafka import Producer
import pandas as pd
import numpy as np
# from faker import Faker
import time
import json
import random

def getStockDataVec():
    df = pd.read_csv("./agents/data/full_data.csv")
    df = df[['open_time', 'close', 'volume', 'count', 'taker_buy_volume', 'symbol']]
    df.set_index('open_time', inplace=True)
    symbol_mapping = {'ADABUSD': 0, 'ETHBUSD': 1, 'BNBBUSD': 2, 'BTCBUSD': 3, 'DOTBUSD': 4}
    df['symbol'] = df['symbol'].map(symbol_mapping).fillna(df['symbol'])
    return df

def getState(sub_data):
    sub_data = sub_data.to_dict()
    return json.dumps(sub_data).encode('utf-8')

def producer():
    # Khởi tạo Kafka producer
    bootstrap_servers = 'localhost:9091'
    topic = 'trading3'
    producer = Producer({'bootstrap.servers': bootstrap_servers})

    data = getStockDataVec()
    index = data.index
   

    # Gửi dữ liệu giả lập đến Kafka
    for i in range(len(data/5)):
        stream_data = getState(data.loc[index[i]])
        print("Thu tu: ", i)
        print(stream_data)
        # print(stream_data.tobytes())
        # print(np.frombuffer(stream_data.tobytes(), dtype=np.float64))
        send_data_to_kafka(producer, topic, stream_data)

        # Thời gian chờ giữa các lần gửi dữ liệu
        time.sleep(5)
        

def send_data_to_kafka(producer, topic, stream_data):
    producer.produce(topic, value=stream_data)
    producer.flush()



In [2]:
producer()

Thu tu:  0
b'{"close": {"1682899200000": 5.88}, "volume": {"1682899200000": 948.78}, "count": {"1682899200000": 19}, "taker_buy_volume": {"1682899200000": 130.22}, "symbol": {"1682899200000": 4}}'


KeyboardInterrupt: 

In [43]:
data = getStockDataVec()
index = data.index
stream_data = getState(data.loc[index[1]])

In [72]:
import json
sub_data = data.loc[index[1]].to_dict()
json.dumps(sub_data).encode('utf-8')

b'{"close": {"1682899200000": 5.88}, "volume": {"1682899200000": 948.78}, "count": {"1682899200000": 19}, "taker_buy_volume": {"1682899200000": 130.22}, "symbol": {"1682899200000": 4}}'

In [25]:
pd.array(data.loc[index[1]])

TypeError: Cannot pass DataFrame to 'pandas.array'

### test Spark

In [8]:

import findspark
findspark.init()

import pyspark
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
sc = pyspark.SparkContext()


In [13]:
  
from queue import Queue
  
# Initializing a queue
q = []
q.append('a')
q.append('b')
q.append('c')
  
# qsize() give the maxsize 
# of the Queue 

print(len(q) ==5) 
print(q)
for i in range(50):
    print(f'===================={i}=============')
    if len(q)==5:
        q.pop(0)
        q.append(i)
    else:
        q.append(i)
    print(q)

False
['a', 'b', 'c']
====================0=============
['a', 'b', 'c', 0]
====================1=============
['a', 'b', 'c', 0, 1]
====================2=============
['b', 'c', 0, 1, 2]
====================3=============
['c', 0, 1, 2, 3]
====================4=============
[0, 1, 2, 3, 4]
====================5=============
[1, 2, 3, 4, 5]
====================6=============
[2, 3, 4, 5, 6]
====================7=============
[3, 4, 5, 6, 7]
====================8=============
[4, 5, 6, 7, 8]
====================9=============
[5, 6, 7, 8, 9]
====================10=============
[6, 7, 8, 9, 10]
====================11=============
[7, 8, 9, 10, 11]
====================12=============
[8, 9, 10, 11, 12]
====================13=============
[9, 10, 11, 12, 13]
====================14=============
[10, 11, 12, 13, 14]
====================15=============
[11, 12, 13, 14, 15]
====================16=============
[12, 13, 14, 15, 16]
====================17=============
[13, 14, 15, 16, 17]
=======

In [ ]:
q.full()

True

In [10]:
q.put('d')


In [11]:
q.full()

False

In [1]:
import torch

print(torch.__version__)

2.0.1+cpu


In [20]:
from binance.client import Client
import time
def create_train_data(data):
    listData = []
    listData.append(round(float(data[4]), 3))
    listData.append(round(float(data[5]), 3))
    listData.append(round(float(data[8]), 3))
    listData.append(round(float(data[9]), 3))

    if data[12] == "BTCUSDT":
        listData.append(0)
    elif data[12] == "ETHUSDT":
        listData.append(1)
    elif data[12] == "BNBUSDT":
        listData.append(2)
    elif data[12] == "XRPUSDT":
        listData.append(3)
    else:
        listData.append(4)

    return listData

trading_pairs = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "XRPUSDT", "LTCUSDT"]
api_key = '38a8NZYdoKTqdQBvztjSDVI0kZUSuvA6tC0LDKohH5hPfE5S6QTxJWIe9FMZFj2q'
api_secret = 'bTEGJyrHWKpPHSxfF3nXAu8ruZsbcXqYwlKnHDiI4mRKRNqH6VbB9ILsmTbQRYXc'
client = Client(api_key, api_secret)


data2 = [] # contain data with windowsize =5
while True:

    sub_data =[] # contain data for each day
    for pair in trading_pairs:
        pair_data = client.get_historical_klines(pair, Client.KLINE_INTERVAL_3MINUTE, "3 min ago UTC")
        pair_data[0].append(pair)
        sub_data.append(create_train_data(pair_data[0]))

    if(len(data2)==10):
        data2.pop(0)
        data2.append(sub_data)
        # send_data = json.dumps(data).encode('utf-8')
        # send_data_to_kafka(producer, topic, send_data)
        # print(send_data)
        break
        
    else:
        data2.append(sub_data)
    time.sleep(1)

In [21]:
data2

[[[30189.0, 18.298, 289.0, 0.257, 0],
  [1913.4, 588.767, 386.0, 222.286, 1],
  [242.1, 80.434, 97.0, 22.204, 2],
  [0.735, 216120.0, 174.0, 144577.0, 3],
  [92.36, 6226.47, 150.0, 168.488, 4]],
 [[30189.0, 18.456, 296.0, 0.364, 0],
  [1913.4, 595.787, 391.0, 222.296, 1],
  [242.1, 89.112, 103.0, 22.553, 2],
  [0.734, 227414.0, 190.0, 144577.0, 3],
  [92.36, 6226.47, 150.0, 168.488, 4]],
 [[30188.95, 18.585, 307.0, 0.367, 0],
  [1913.39, 604.169, 398.0, 225.793, 1],
  [242.1, 97.69, 107.0, 22.553, 2],
  [0.734, 229400.0, 194.0, 146204.0, 3],
  [92.37, 6229.684, 152.0, 171.702, 4]],
 [[30188.0, 19.446, 330.0, 0.371, 0],
  [1913.21, 623.733, 419.0, 225.801, 1],
  [242.1, 112.092, 124.0, 26.743, 2],
  [0.734, 233034.0, 206.0, 146502.0, 3],
  [92.36, 6232.184, 154.0, 171.702, 4]],
 [[30187.44, 20.319, 380.0, 0.647, 0],
  [1913.21, 651.805, 461.0, 253.383, 1],
  [242.1, 132.911, 132.0, 46.216, 2],
  [0.734, 238227.0, 214.0, 146787.0, 3],
  [92.35, 6236.282, 157.0, 171.702, 4]],
 [[30187.45,

In [8]:
from confluent_kafka import Producer
import pandas as pd
import numpy as np
# from faker import Faker
import time
import json
import random

In [14]:
def send_data_to_kafka(producer, topic, stream_data):
    producer.produce(topic, value=stream_data)
    producer.flush()
 # Initialize producer
bootstrap_servers = 'localhost:9091'
topic = 'nguyentri'
producer = Producer({'bootstrap.servers': bootstrap_servers},)

In [68]:
send_data = json.dumps(data).encode('utf-8')
send_data_to_kafka(producer, topic, send_data)
print("oke data")

oke data


In [63]:
send_data2 = json.dumps(data2).encode('utf-8')
send_data_to_kafka(producer, topic, send_data2)
print("oke data2")

oke data2


In [13]:
np.array(data).reshape(1,250)

array([[3.016589e+04, 1.694710e+02, 3.548000e+03, 7.691700e+01,
        0.000000e+00, 1.915460e+03, 2.849366e+03, 3.288000e+03,
        1.009989e+03, 1.000000e+00, 2.422000e+02, 2.471344e+03,
        1.135000e+03, 1.006388e+03, 2.000000e+00, 7.310000e-01,
        2.625494e+06, 1.999000e+03, 1.436714e+06, 3.000000e+00,
        9.220000e+01, 8.939336e+03, 1.374000e+03, 4.056845e+03,
        4.000000e+00, 3.017459e+04, 1.732500e+02, 3.681000e+03,
        7.964700e+01, 0.000000e+00, 1.916210e+03, 2.908055e+03,
        3.363000e+03, 1.043765e+03, 1.000000e+00, 2.423000e+02,
        2.549822e+03, 1.169000e+03, 1.074002e+03, 2.000000e+00,
        7.320000e-01, 2.712969e+06, 2.053000e+03, 1.498003e+06,
        3.000000e+00, 9.221000e+01, 8.941156e+03, 1.377000e+03,
        4.058665e+03, 4.000000e+00, 3.017460e+04, 1.737380e+02,
        3.693000e+03, 7.966300e+01, 0.000000e+00, 1.916260e+03,
        2.988733e+03, 3.436000e+03, 1.107925e+03, 1.000000e+00,
        2.422000e+02, 2.557065e+03, 1.17

In [45]:
data

[[[30300.01, 12.53, 723.0, 3.354, 0],
  [1934.72, 106.28, 316.0, 43.397, 1],
  [250.3, 177.484, 171.0, 32.873, 2],
  [0.722, 528558.0, 409.0, 196453.0, 3],
  [94.69, 296.806, 137.0, 191.537, 4]],
 [[30300.01, 12.543, 725.0, 3.356, 0],
  [1934.71, 109.416, 320.0, 46.526, 1],
  [250.2, 177.538, 172.0, 32.873, 2],
  [0.722, 528855.0, 410.0, 196453.0, 3],
  [94.69, 296.879, 138.0, 191.61, 4]],
 [[30300.01, 12.605, 727.0, 3.406, 0],
  [1934.72, 109.469, 321.0, 46.579, 1],
  [250.3, 178.29, 174.0, 32.892, 2],
  [0.722, 544553.0, 421.0, 196453.0, 3],
  [94.69, 297.348, 140.0, 192.079, 4]],
 [[30300.01, 12.621, 729.0, 3.422, 0],
  [1934.72, 109.469, 321.0, 46.579, 1],
  [250.3, 178.429, 175.0, 33.031, 2],
  [0.722, 544553.0, 421.0, 196453.0, 3],
  [94.69, 297.348, 140.0, 192.079, 4]],
 [[30300.01, 12.622, 731.0, 3.423, 0],
  [1934.71, 109.583, 323.0, 46.579, 1],
  [250.2, 178.829, 176.0, 33.031, 2],
  [0.722, 544570.0, 422.0, 196470.0, 3],
  [94.68, 297.947, 142.0, 192.46, 4]],
 [[30300.01, 12

1

In [2]:
import torch
print(torch.__version__)

2.0.1+cpu
